In [6]:
# Importar librerias
import numpy as np
import pandas as pd
import openai

import os
from dotenv import find_dotenv, load_dotenv

# **Audio to Text**

In [ ]:
# Transcribir el texto
import whisper

model = whisper.load_model("small")
result = model.transcribe("asertividad.mp3")

with open("transcripcion.txt", "w") as f:
    f.write(result["text"])

In [ ]:
# Separar por punto cada línea
with open("transcripcion.txt", "r") as f:
    line = f.read().strip()
    lines = line.split(".")

In [ ]:
# Escibir un csv con cada una de las líneas
file_name = "chatbotAsertivo.csv"
with open(file_name, "w") as f:
    f.write("|texto|\n")
    for line in lines:
        f.write("|" + line.strip() + "|" + "\n")

# **Embeddings**

**API Keys**

In [11]:
# Definimos la API Key para vincular el cuaderno con nuestra cuenta de OpenAI
load_dotenv(find_dotenv())
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("API_BASE")

## **Embeddings**

In [9]:
# Obtener el embedding the un texto
def get_embedding(text, model="text-embedding-ada-002"):
   if (repr(text) != 'nan'):
      text = text.replace("\n", "")
      return openai.Embedding.create(input = text, model=model)['data'][0]['embedding']

In [8]:
# Texto raw to dataframe y guardar archivo pickle
def embed_text(path="texto.csv"):
    conocimiento_df = pd.read_csv(path, sep="|", encoding='latin-1', usecols=['texto']).dropna()
    conocimiento_df['Embedding'] = conocimiento_df['texto'].apply(lambda x: get_embedding(x))
    conocimiento_df.to_pickle('resources/embeddings.pkl')
    return conocimiento_df

In [7]:
# Obtener embedding previo u obtenerlo desde cero 
# texto_emb = pd.read_csv('embeddings.csv') # csv
texto_emb = pd.read_pickle('resources/embeddings.pkl') # pkl

# file_name = "resources/asertividad.csv"
# texto_emb = embed_text(file_name) # Obtener de cero

In [13]:
# Obtener la similitud entre dos vectores 
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

## **Búsqueda**

In [2]:
def buscar(busqueda, datos, n_resultados=5):
    busqueda_embed = get_embedding(busqueda)

    datos["Similitud"] = datos['Embedding'].apply(lambda x: cosine_similarity(x, busqueda_embed))
    datos = datos.sort_values("Similitud", ascending=False)
    return datos.iloc[:n_resultados][["texto", "Similitud", "Embedding"]]

In [25]:
# Búsqueda
message = "quiero decirle no a algo que no me gusta a una persona que quiero mucho, pero como lo hago?"
resultados_df = buscar(to_search, texto_emb, n_resultados=5)

# **ChatBot** 

In [38]:
# Definir la orden predeterminada para el LLM
template = """
Eres la mejor coach de mejora personal, para personas de entre 18 y 27 años.
Te voy a compartir una solicitud de una de estas personas que pertenecen al público objetivo.
Tu te encargarás de ofrecer la mejor respuesta acorde a qué dicen las mejores coach o incluso psicólogas respecto a la temática relacionada.
Además tendrás que seguir cuidadosamente las siguientes TODAS las reglas de a continuación:

1/ La respuesta debe ser demasiado similar o incluso idéntica a qué dicen las expertas en el tema, 
esto en función de su forma de hablar, sus argumentos lógicos y cualquier otro detalle que identifiques.

Debajo se encuentra la solicitud:
{message}

Ahora te muestro a continuación qué dicen las expertas acerca del tema, con lo cual puedas basar tu respuesta:
{experts}

Por favor, escribe cómo le responderías a esta persona que ha acudido a ti como coach:
"""

In [17]:
# Obtener arreglo de similitudes y pasarlo a tipo String
arr_similitudes = resultados_df['texto'].to_numpy()
str_similitudes = np.array2string(arr_similitudes)

In [41]:
# Obtener el template con los datos a cargar para el LLM
temp_to_load = template.format(message=message, experts=str_similitudes)

In [46]:
response = openai.ChatCompletion.create(
    model='gpt-4',
    messages=[
        {'role': 'user', 'content': temp_to_load},
    ],
    stream=True,
    allow_fallback=True
)

for chunk in response:
    print(chunk.choices[0].delta.get("content", ""), end="", flush=True)

Entiendo que estás en una situación delicada y me alegra que hayas decidido buscar ayuda. Aquí te dejo algunos consejos que podrían serte útiles.

Primero, recuerda que tus sentimientos y opiniones son válidos y merecen ser expresados. No necesitas estar de acuerdo con todo lo que la otra persona piensa o hace para mantener una relación armoniosa. De hecho, nuestras diferencias pueden coexistir en respeto y paz. No estamos obligados a pensar igual, opinar igual o tener los mismos gustos.

Puedes usar frases como "opino diferente", "estoy en desacuerdo", "no estoy de acuerdo pero lo respeto", "respeto pero no lo comparto". Estas expresiones son una manera asertiva de comunicar tu desacuerdo sin invalidar o minimizar la perspectiva de la otra persona

## Diferencias embeddings y chatgpt

En respuesta a: "quiero decirle no a algo que no me gusta a una persona que quiero mucho, pero como lo hago?"

**Embeddings**

Entiendo que estás en una situación delicada y me alegra que hayas decidido buscar ayuda. Aquí te dejo algunos consejos que podrían serte útiles.

Primero, recuerda que tus sentimientos y opiniones son válidos y merecen ser expresados. No necesitas estar de acuerdo con todo lo que la otra persona piensa o hace para mantener una relación armoniosa. De hecho, nuestras diferencias pueden coexistir en respeto y paz. No estamos obligados a pensar igual, opinar igual o tener los mismos gustos.

Puedes usar frases como "opino diferente", "estoy en desacuerdo", "no estoy de acuerdo pero lo respeto", "respeto pero no lo comparto". Estas expresiones son una manera asertiva de comunicar tu desacuerdo sin invalidar o minimizar la perspectiva de la otra persona.

**ChatGPT**

Decir "no" a algo que no te gusta a una persona a la que aprecias puede ser un desafío, pero es importante establecer límites y comunicarte de manera honesta y respetuosa. Aquí hay algunos pasos que puedes seguir:

1. **Escoge el momento adecuado:** Encuentra un momento en el que ambos estén relajados y tengan tiempo para hablar sin interrupciones. Evita hacerlo cuando estén enojados o tensos.

2. **Prepárate:** Antes de hablar, piensa en tus razones para decir "no" y en cómo puedes explicar tu posición de manera clara y calmada.

3. **Sé directo pero amable:** Comunica tu negativa de manera directa y honesta, pero hazlo de una manera amable y considerada. Evita rodeos o ambigüedades que puedan causar malentendidos.

4. **Usa el "yo" en lugar del "tú":** Enfoca la conversación en tus propios sentimientos y necesidades en lugar de criticar o culpar a la otra persona. Por ejemplo, en lugar de decir "tú siempre haces esto", podrías decir "yo me siento incómodo cuando..."

5. **Explica tus razones:** Si es apropiado, brinda una explicación de por qué estás diciendo "no". Ser transparente ayudará a la otra persona a entender tu punto de vista.

6. **Ofrece alternativas:** Si es posible, sugiere alternativas que podrían funcionar mejor para ambas partes. Esto demuestra que estás comprometido en encontrar soluciones y que valoras la relación.

7. **Escucha su respuesta:** Después de expresar tu posición, permite que la otra persona comparta sus pensamientos y sentimientos al respecto. Escuchar activamente muestra que te importa su perspectiva.

8. **Mantén la calma:** Es posible que la otra persona no tome tu respuesta de manera positiva inicialmente. Mantén la calma y evita caer en discusiones o confrontaciones.

9. **Reafirma tu aprecio:** Después de expresar tu negativa, reafirma tu aprecio por la persona. Hazles saber que tu decisión no afecta tus sentimientos hacia ellos.

10. **Da tiempo:** A veces, las personas necesitan tiempo para procesar la información y tus razones. Dale espacio a la otra persona para pensar y responder en su propio tiempo.

Recuerda que es natural sentir ansiedad o preocupación al decir "no", pero establecer límites saludables es esencial para mantener relaciones equilibradas. Una comunicación abierta y respetuosa es clave para mantener la conexión y el entendimiento mutuo. [ChatGPT Response](https://chat.openai.com/share/07d92e7c-0202-41dc-ad1e-5699bd5652b5)

**Análisis**
* Podemos apreciar que ChatGPT da respuestas que pueden sonar **comunes**. 
* Además acostumbra ser por el momento una respuesta mucho **más larga** que la ofrecida con el embedding bajo data previa.
* Por último, me gustaría recalcar el hecho en el que por el momento la respuesta del embedding es mucho más directa a la **acción** en palabras concretas.